In [2]:
import numpy as np
import pandas as pd

import time

import matplotlib.pyplot as plt

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials 
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id='0c7c294438794162b901ce0554a38703', client_secret='cecd70a59c1344f7a7d8d7cbec4fe6bb'))

#uncomment above and replace CLIENT_ID and CLIENT_SECRET with the id and secret you get from creating your app with spotify (going through the "Getting Started" instructions in the Spotify Web API documentation)

from sklearn.model_selection import train_test_split
import math

# Spotify Integration

In [3]:
discussions_df = pd.read_csv('data/Discussions.csv', header=0, encoding='utf-8')
ratings_df = pd.read_csv('data/Ratings.csv', header=0, encoding='utf-8')

# creating new columns and intializing each value to 0
discussions_df['EnergyAvg'] = 0
discussions_df['ValenceAvg'] = 0
discussions_df['AcousticnessAvg'] = 0
discussions_df['InstrumentalnessAvg'] = 0

display(discussions_df)

FileNotFoundError: [Errno 2] No such file or directory: 'data/Discussions.csv'

In [ ]:
for ind in range(len(discussions_df)):
    results = spotify.album_tracks(discussions_df['SpotifyID'][ind]) # getting tracks from current album via SpotiPy

    # filtering track ids into list
    ids = []
    for track in results['items']:
        ids.append(track['id'])
    numTracks = len(ids)

    # getting features from each track in ids via SpotiPy
    features = spotify.audio_features(ids)
    for feature in features['audio_features']:
        # calculating and storing average in df
        discussions_df['EnergyAvg'][ind] += feature['energy'] / numTracks
        discussions_df['ValenceAvg'][ind] += feature['valence'] / numTracks
        discussions_df['AcousticnessAvg'][ind] += feature['acousticness'] / numTracks
        discussions_df['InstrumentalnessAvg'][ind] += feature['instrumentalness'] / numTracks
    time.sleep(3) # sleep with arbitrary time of 3 seconds for circumventing rate limit
    
display(discussions_df)

# Your Turn
Fill out the the database with audio features (watch out for 429 errors!) and run some of the filtering techniques we've covered on it.

Content filtering is a great fit here

also be cognisant of the fact that audio features are numerical values, and thus can be leveraged in weighted sums, numerical analysis, and any number of data science techniques.

and like mentioned in the content filtering slides, feel free to go above and beyond with any external resoruces (like RateYourMusic, AlbumOfTheYear, etc.) you'd like!

In [1]:
# TODO: integrate Spotify with your filtering code
discussions_df['energy'] = 0
for ind in range(len(discussions_df)):
    try:
        results = spotify.album_tracks(discussions_df['SpotifyID'][ind]) # getting tracks from current album via SpotiPy

        # filtering track ids into list
        ids = []
        for track in results['items']:
            ids.append(track['id'])
        numTracks = len(ids)
        features = spotify.audio_features(ids)
        for feature in features:
            # calculating and storing average in df
        
            discussions_df['energy'][ind] += (feature['speechiness'] + feature['Rating'])
    except:
        pass
    
lengthy = discussions_df[['DiscussionID','AlbumName', 'energy', 'AvgRating']]

ratings_df['energy'] = lengthy['energy']

X = ratings_df.drop(columns=['energy'])
y = ratings_df['energy']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train['energy'] = y_train

merged_df = pd.merge(lengthy, X_train, on='DiscussionID')

display(merged_df[["DiscussionID", "AlbumName", "energy_x", "Rating"]])
y_pred = []
for idx, row in X_test.iterrows():
    # Get the decade of the current album
    album_decade = discussions_df.loc[discussions_df['DiscussionID'] == row['DiscussionID'], 'energy'].iloc[0]
    
    # Filter merged_df to only include discussions attended by the user and in the same decade
    smaller_df = merged_df[(merged_df['MemberID'] == row['MemberID'])]
    
    # Calculate the average rating for discussions in the same decade
    try:
        avg_rating = smaller_df['energy_y'].mean() if not smaller_df.empty else X_train['energy_y'].mean()
    except:
        pass
    y_pred.append(float(np.round(avg_rating, 0)))

# Calculate RMSE
y_pred = np.array(y_pred)
rmse = np.sqrt(np.mean((y_pred - y_test)**2))
print("RMSE:", rmse)
